In [1]:
%run ../utils/commonImports.py
%run ../utils/tradingImports.py
%matplotlib inline

from utils.common import *

# Load Data

In [2]:
ohlc = pd.read_csv('D:\\Dropbox\\My work\\krypl-project\\dataCleaned\\poloniex\\5min\\USDT_BTC_5min_2016-01-01_2017-12-31.tsv', sep='\t')\
    .query("date >= '2017-01-01'")\
    .sort_values('date').reset_index().drop('index', axis=1)

In [3]:
dataRoot = 'D:\\Dropbox\\My work\\krypl-project\\dataFeatures\\poloniex\\5min'
fileName = 'USDT_BTC_5min_2016-01-01_2017-12-31.tsv'
file = '{dataRoot}\\{fileName}'.format(dataRoot=dataRoot, fileName=fileName)

usdtBtc = pd.read_csv(file, sep='\t').query("date >= '2017-01-01'")\
    .sort_values('date').reset_index().drop('index', axis=1)
usdtBtc['timestamp'] = usdtBtc.date.apply(strTimeToTimestamp).astype(int)

trainRatio = 0.7
trainSize = int(usdtBtc.shape[0] * trainRatio)
usdtBtcTrain = usdtBtc.iloc[:trainSize]
usdtBtcTest = usdtBtc.iloc[trainSize:].reset_index().drop('index', axis=1)
ohlcTrain = ohlc.iloc[:trainSize]
ohlcTest = ohlc.iloc[trainSize:]

usdtBtcManagerTrain = CurrencyDataManager(usdtBtcTrain['close'], usdtBtcTrain[['close']])
usdtBtcManagerTest = CurrencyDataManager(usdtBtcTest['close'], usdtBtcTest[['close']])
wallet = {'usdt': 1000}
contractPair = ContractPair.new('usdt', 'btc')

# Explore

In [ ]:
%matplotlib notebook

In [ ]:
usdtBtcR.columns

In [ ]:
usdtBtcR.iloc[190:, :]

In [ ]:
step = 361
day = 288
i = day * step
j = i + day
usdtBtcR = usdtBtc.iloc[i:j, :]

paper_rc = {'lines.linewidth': 0.5, 'lines.markersize': 1}                  
sns.set_context("paper", rc = paper_rc) 

fig, ax = plotCandles(ohlc.iloc[i:j, :])
ax2 = ax.twinx()
sns.pointplot(usdtBtcR.timestamp.values, usdtBtcR['vol36'].values, ax=ax2)
sns.pointplot(usdtBtcR.timestamp.values, usdtBtcR['zscore_36_return'].values, ax=ax2, color='green')

set_date_axis(usdtBtcR['timestamp'], ax, fig)

# Implementation

In [21]:
from copy import deepcopy

class CustomStrategy:
    def __init__(self, exchange, dataManager, historyLen, contractPair, tradeSize, willingLoss, 
                 distanceFromMaxThreshold1, distanceFromMaxThreshold2, targetProfit):
        self.exchange = exchange
        self.walletStart = deepcopy(self.exchange.wallet)
        self.dataManager = dataManager
        self.historyLen = historyLen
        self.contractPair = contractPair
        self.opened = False
        self.tradeSize = tradeSize
        self.willingLoss = willingLoss
        self.distanceFromMaxThreshold1 = distanceFromMaxThreshold1
        self.distanceFromMaxThreshold2 = distanceFromMaxThreshold2
        self.targetProfit = targetProfit
    
    
    def isRisky(self, priceBought, actualPrice):
        return self.currentReturn(priceBought, actualPrice) < -self.willingLoss 
    
    
    def currentReturn(self, priceBought, actualPrice):
        feePart = (1 - self.exchange.fee) ** 2
        returnPart = (actualPrice * feePart) / priceBought
        return returnPart - 1
    
    
    def sellWhole(self, price):
        self.opened = False
        amount = self.exchange.balance(self.contractPair['tradeContract'])
        self.exchange.sell(self.contractPair, amount, price)
    
    
    def buy(self, price):
        amount = self.tradeSize / price
        self.exchange.buy(self.contractPair, amount, price)
        self.opened = True
        
        fee = self.exchange.fee
        breakEvenPrice = price / ((1 - fee) ** 2)
#         print('price', price, 'breakEven', breakEvenPrice)
        return breakEvenPrice
        
    
    def isTargetSatisffied(self, priceBought, price):
        return self.currentReturn(priceBought, price) >= self.targetProfit
    
    
    def currentTime(self):
        return self.dataManager.time
    
    def timeExceeded(self, timeBought):
        return (self.currentTime() - timeBought) > self.historyLen
    
    def trade(self):
        
        wasBellow = False
        maxPrice = -1
        while self.dataManager.has_tick():            
            history, price = self.dataManager.tick(self.historyLen)
            if history.shape[0] < self.historyLen:
                continue

            closeMax = history[0].max()
            distanceFromMax = (closeMax / price) - 1
            distanceFromMax2 = (maxPrice / price) - 1
#             print(history.shape[0], distanceFromMax, closeMax, price)
            if not self.opened and not wasBellow and distanceFromMax >= self.distanceFromMaxThreshold1:
                wasBellow = True
                maxPrice = closeMax
            elif not self.opened and wasBellow and distanceFromMax2 <= self.distanceFromMaxThreshold2:
#                 print(self.dataManager.time, 'openning trade: ', price, end='\t')
                breakEvenPrice = self.buy(price)
                priceBought = price
                timeBought = self.currentTime()
            elif self.opened and (self.isRisky(priceBought, price)):
#                 print(self.dataManager.time, 'stopLoss: ', price, self.currentReturn(breakEvenPrice, price) * 100)
                self.sellWhole(price)
                wasBellow = False
            elif self.opened and self.isTargetSatisffied(breakEvenPrice, price):
#                 print(self.dataManager.time, 'profit: ', price, self.currentReturn(breakEvenPrice, price) * 100)
                self.sellWhole(price)
                wasBellow = False
        
        if self.opened:
            self.sellWhole(price)

                
    def stats(self, contractName):
        return Statistics(contractName, self.walletStart[contractName]).evaluate(self.exchange.transactions)

# Learning

In [13]:
dataManager = deepcopy(usdtBtcManagerTrain)
exchange = BackTestExchange(dataManager, deepcopy(wallet), 0.0025)
strategy = CustomStrategy(exchange, dataManager, 1*288, contractPair, 100, willingLoss=0.05, 
                                   distanceFromMaxThreshold1=0.04, distanceFromMaxThreshold2=0.035, targetProfit=0.02)
strategy.trade()
strategy.stats('usdt').report()

13444 profit:  1039.4999999 2.463551993513713
16718 profit:  1199.47833305 2.0214320151324072
20349 profit:  1205.0 2.5025734012354306
30878 profit:  1255.60001014 2.2278796565172065
34218 profit:  1433.99999999 2.1628732050881894
35763 profit:  1635.41150036 2.13703877109761
36578 profit:  1652.53807967 2.1907911200630847
37851 profit:  1845.0 2.078460526076431
38442 profit:  1854.94999999 2.026108041843555
39219 profit:  1841.0 2.1637027180708035
41710 profit:  2379.94999999 2.4451999765735533
41741 profit:  2430.99999994 2.547561370012641
41844 profit:  2427.92899993 2.617456115452721
44909 profit:  2630.0 2.0843837285223765
45984 profit:  2850.0 2.074369447440927
58217 profit:  2741.84999985 2.1653800514856325
60018 profit:  2770.0 2.137940483545764
60929 profit:  2790.0 2.0003189444017355
61887 profit:  2806.0 2.0477188309374394
63773 profit:  3424.42 2.0329854443271733
65640 profit:  4357.0 2.02520729527631
67427 profit:  4105.99999998 2.1220451266882767
70835 profit:  4580.0 2.0

,usdt
startAmount,"1,000.0000"
numberOfTrades,43.0000
totalProfit,-53.5407
avgProfit,-1.2451
winPercentage,53.4884
avgWinTrade,2.7066
avgLossTrade,-5.7896
profitFactor,0.5376
maxDrawdown,5.3670


In [ ]:
exchange.wallet

In [32]:
import time, sys
DAY = 288

allStats = None
print('historyLen willingLoss distanceFromMaxThreshold1 distanceFromMaxThreshold2 targetProfit')
for historyLen in [DAY, 2*DAY, 3*DAY, 7*DAY]:
    for willingLoss in [.01, 0.02, .03, .05, .1, .2, .5]:
        distanceThresholds = [.01, .02, .03, .04, .05, 0.06, .07, .08]
        for distanceFromMaxThreshold1 in distanceThresholds:
            for distanceFromMaxThreshold2 in [x for x in distanceThresholds if x <= distanceFromMaxThreshold1]:
                for targetProfit in [.01, .02, 0.03, 0.04, .05, .07, .1, .2, .3]:
                    s = "\r{} {} {} {} {}".format(historyLen, willingLoss, distanceFromMaxThreshold1, 
                                                  distanceFromMaxThreshold2, targetProfit)
                    sys.stdout.write(s)
                    sys.stdout.flush()

                    start_time = time.time()
                    dataManager = deepcopy(usdtBtcManagerTrain)
                    exchange = BackTestExchange(dataManager, deepcopy(wallet), 0.0025)
                    strategy = CustomStrategy(exchange, dataManager, historyLen, contractPair, 100, willingLoss=willingLoss, 
                                              distanceFromMaxThreshold1=distanceFromMaxThreshold1, 
                                              distanceFromMaxThreshold2=distanceFromMaxThreshold2, 
                                              targetProfit=targetProfit)
                    strategy.trade()
                    stats = strategy.stats('usdt').report().transpose().reset_index().rename(columns={'index': 'contract'})
                    stats['historyLen'] = historyLen
                    stats['willingLoss'] = willingLoss
                    stats['distanceFromMaxThreshold1'] = distanceFromMaxThreshold1
                    stats['distanceFromMaxThreshold2'] = distanceFromMaxThreshold2
                    stats['targetProfit'] = targetProfit
                    if allStats is None:
                        allStats = stats
                    else:
                        allStats = allStats.append(stats)
                    sys.stdout.write(s + "\t %.2f seconds" % (time.time() - start_time))
                    sys.stdout.flush()

allStats.to_csv('D:\\Dropbox\\My work\\krypl-project\\result\\customStrategy.tsv', index=False, sep='\t')


historyLen willingLoss distanceFromMaxThreshold1 distanceFromMaxThreshold2 targetProfit
288 0.01 0.03 0.01 0.05	 0.38 seconds

KeyboardInterrupt: 

# Look on transactions

In [ ]:
from matplotlib.finance import candlestick_ohlc
import matplotlib.dates as mdates

import time
import datetime as dt

def timestampToDate(timestamp):
    return dt.datetime.fromtimestamp(int(timestamp))

def strTimeToTimestamp(strTime):
    return dt.datetime.strptime(strTime, "%Y-%m-%d %H:%M:%S").timestamp()


def calculateMA(history, col, timePeriod):
        return calculateTalib('MA', {col: history[col].values}, {'timeperiod': timePeriod})


def plotCandles(df, currencyPair):
    DATA = df[['open', 'high', 'low', 'close', 'volume', 'date', 'timestamp']].copy()
    DATA = DATA.reset_index()
    DATA["dateStr"] = DATA["date"]
    DATA["date"]  = DATA["timestamp"].apply(timestampToDate).apply(mdates.date2num)

    plt.figure(figsize=(16, 8))
    ax = plt.gca()
    candlestick_ohlc(ax, DATA.values, width=.6, colorup='#53c156', colordown='#ff1717')

    ax.xaxis_date()
    
    xDates = DATA['dateStr']
    step = int(xDates.shape[0] / 20) + 1
    xDates[[i for i in range(xDates.shape[0]) if i % step != 0]] = ''
    
    plt.xticks(range(DATA.shape[0]), DATA['dateStr'], rotation=45)
    plt.title(currencyPair, fontproperties=titleFont)
    

def plotTransaction(transaction, i):
    print(transaction)
    color = 'k' if transaction['type'] == Transaction.BUY else 'm'
    print(color)
    plt.scatter(i, transaction['price'], s=200, c=color)
    

def plotMax(mmax, i):
    plt.scatter(i, mmax, s=200, c='b')

    
def indexRange(i, offsetBack, offsetForward, length):
    start = i - offsetBack if i - offsetBack > 0 else 0
    end = i + offsetForward if i + offsetForward < length else length-1
    return start, end
    
def plotTransactionWithSurroundings(prices, timePeriod, transaction):
    i = transaction['timestamp']
    start,end = indexRange(i, 40, 20, prices.shape[0])
    selected = prices.iloc[start:end, :].reset_index()
    
    plotCandles(selected, '')
    plotTransaction(transaction, 39)
    
    mmax = prices.iloc[start:i, :]['close'].max()
#     plotMax(mmax, 39)
    

In [ ]:
len(strategy.exchange.transactions)

In [ ]:
def plotTransactions(ohlc, transactions):
    ohlcMaxI = ohlc.shape[0]
    buys = transactionsToPlot(transactions, Transaction.BUY)
    sells = transactionsToPlot(transactions, Transaction.SELL)
    
    firstTransaction = int(min(min(buys[:, 0]), min(sells[:, 0])))
    firstTransaction = firstTransaction - 40 if firstTransaction - 40 > 0 else 0
    
    lastTransaction = int(max(max(buys[:, 0]), max(sells[:, 0])))
    lastTransaction = lastTransaction + 40 if lastTransaction + 40 < ohlcMaxI else ohlcMaxI
    
    fig, ax = plotCandles(ohlc.iloc[firstTransaction:lastTransaction, :])
    
    buyScatter = plt.scatter(buys[:, 0] - firstTransaction, buys[:, 1], s=200, c='g', label='buy')
    sellScatter = plt.scatter(sells[:, 0] - firstTransaction, sells[:, 1], s=200, c='m', label='sell')

    plt.legend(handles=[buyScatter, sellScatter], loc='upper left')

In [ ]:
%matplotlib notebook
plotTransactions(ohlcTrain, strategy.exchange.transactions[50:])

In [ ]:
t = 70

transaction = strategy.exchange.transactions[t]
plotTransactionWithSurroundings(ohlcTrain, strategy.historyLen, transaction)
transaction = strategy.exchange.transactions[t+1]
plotTransactionWithSurroundings(ohlcTrain, strategy.historyLen, transaction)

In [ ]:
transaction

# Evaluate Distance From Max

In [ ]:
allStats = pd.read_csv('D:\\Dropbox\\My work\\krypl-project\\result\\distanceFromMaxWithTimeRestriction.tsv',sep='\t')

In [ ]:
allStats.query('totalProfit > 90 and winPercentage > 70').sort_values('numberOfTrades', ascending=False)

In [ ]:
dataManager = deepcopy(usdtBtcManagerTest)
exchange = BackTestExchange(dataManager, deepcopy(wallet), 0.0025)
strategy = BollingerBandsStrategy(exchange, dataManager, 30, contractPair, 100, willingLoss=0.03, distFromMiddleThreshold=0.02, diffThreshold=0.01)
strategy.trade()
strategy.stats('usdt').report()